## Parte 1

tenemos un mapa de senderismo donce . es un camino, # es bosque y hay cuestas ^, >, v, <. En las cuestas hay que seguir la dirección descendente y buscamos nunca usar un sendero dos veces, cuál es la ruta más larga que se puede hacer?

In [3]:
import networkx as nx

In [4]:
def get_input(filename):
    grid = []
    with open(filename) as file:
        for line in file:
            grid.append([c for c in line.strip()])
    return grid

In [5]:
def get_io(hiking_map):
    icell = (-1, -1)
    ocell = (-1, -1)
    
    for i, c in enumerate(hiking_map[0]):
        if c == '.':
            icell = (0, i)
            break
            
    for i, c in enumerate(hiking_map[-1]):
        if c == '.':
            ocell = (len(hiking_map) - 1, i)
            break
    
    return icell, ocell

In [8]:
def get_graph(hiking_map):
    h = len(hiking_map)
    w = len(hiking_map[0])
    graph = {}
    
    dic_opciones = {'.': [(0, 1), (0, -1), (1, 0), (-1, 0)],
                    '>': [(0, 1)],
                    '<': [(0, -1)],
                    'v': [(1, 0)],
                    '^': [(-1, 0)]}
    
    for i in range(h):
        for j in range(w):
            if hiking_map[i][j] == '#':
                continue
            conexiones = []
            
            cur_opciones = dic_opciones[hiking_map[i][j]]
            for di, dj in cur_opciones:
                if 0 <= i + di and i + di < h and 0 <= j + dj and j + dj < w:
                    if hiking_map[i+di][j+dj] != '#':
                        graph[(i, j)] = graph.get((i, j), []) + [(i + di, j + dj)]
    return graph

In [11]:
def get_max_path(graph, start, end):
    G = nx.from_dict_of_lists(graph, create_using=nx.DiGraph)

    max_path = 0
    for path in nx.all_simple_paths(G, source=start, target=end):
        if len(path) - 1 > max_path:
            max_path = len(path) - 1
    return max_path 

In [12]:
hiking_map = get_input('test.txt')
start, end = get_io(hiking_map)
graph = get_graph(hiking_map)
max_path = get_max_path(graph, start, end)
max_path

94

In [13]:
hiking_map = get_input('input.txt')
start, end = get_io(hiking_map)
graph = get_graph(hiking_map)
max_path = get_max_path(graph, start, end)
max_path

2370

## Parte 2

las cuestas no son tan empinadas así que se pueden seguir en cualquier dirección, obtener el camino más largo sin repetir ruta.

In [25]:
def get_long_graph(hiking_map):
    h = len(hiking_map)
    w = len(hiking_map[0])
    opciones = [(0, 1), (0, -1), (1, 0), (-1, 0)]
    graph = {}
    
    for i in range(h):
        for j in range(w):
            if hiking_map[i][j] == '#':
                continue
            conexiones = []
            
            
            for di, dj in opciones:
                if 0 <= i + di and i + di < h and 0 <= j + dj and j + dj < w:
                    if hiking_map[i+di][j+dj] != '#':
                        graph[(i, j)] = graph.get((i, j), []) + [(i + di, j + dj, 1)]
    return graph

In [26]:
def simplificar(graph):
    simp = True
    while simp:
        simp = False
        for k, v in graph.items():
            if len(v) == 2:
                p1 = v[0]
                p2 = v[1]
                
                graph[p1[0:2]].remove(k + (p1[2],))
                graph[p2[0:2]].remove(k + (p2[2],))
                
                graph[p1[0:2]].append((p2[0], p2[1], p2[2] + p1[2]))
                graph[p2[0:2]].append((p1[0], p1[1], p1[2] + p2[2]))
                del graph[k]
                simp = True
                break
                
    return graph

In [45]:
def get_max_path_w(graph, start, end):
    G = nx.DiGraph()
    for k in graph.keys():
        for v in graph[k]:
            G.add_weighted_edges_from([(k, (v[0], v[1]), v[2])])
            
    max_path = 0
    for i, path in enumerate(nx.all_simple_paths(G, start, end)):
        coste = nx.path_weight(G, path, "weight")
        if max_path < coste:
            max_path = coste
            
    return max_path

In [39]:
hiking_map = get_input('test.txt')
start, end = get_io(hiking_map)
graph = get_long_graph(hiking_map)
simple_graph = simplificar(graph)
max_path = get_max_path_w(simple_graph, start, end)
max_path

154

In [46]:
%%time
hiking_map = get_input('input.txt')
start, end = get_io(hiking_map)
graph = get_long_graph(hiking_map)
simple_graph = simplificar(graph)
max_path = get_max_path_w(simple_graph, start, end)
max_path

CPU times: user 2min 2s, sys: 43.9 ms, total: 2min 2s
Wall time: 2min 3s


6546